# Getting  with usage of AMP-PD through TERRA

Here is the notebook to get hands on 

## Setting up libraries

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

## Initialize workspace variables

In [2]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

print(BILLING_PROJECT_ID)
print(WORKSPACE_NAMESPACE)
print(WORKSPACE_NAME)
print(WORKSPACE_BUCKET)

terra-ed19e231
lw-ucl-hrm-test
Getting started using AMP-PD data
gs://fc-cd759889-2702-4f72-a832-0be756073417


In [3]:
WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2021_v2-5release_0510'
GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical/'

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs'
GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')

BQ_RELEASE_DATASET = 'amp-pd-research.2021_v2-5release_0510'


## Defining handy python functions

In [4]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)


# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Get the data from a query
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_bq(description, link_text, bq_dataset, bq_table=None):
    project, dataset = bq_dataset.split('.', 1)
    if bq_table:
        page_params = {'page': 'table', 'p': project, 'd': dataset, 't': bq_table}
    else:
        page_params = {'page': 'dataset', 'p': project, 'd': dataset}
    
    url = '{}?{}'.format(
        'https://console.cloud.google.com/bigquery',
        urllib.parse.urlencode(page_params))

    display_html_link(description, link_text, url)    

# Utility routine for printing a query before executing it
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

In [5]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_RELEASE_PATH}')


Executing: gsutil -u terra-ed19e231 ls gs://amp-pd-data/releases/2021_v2-5release_0510
gs://amp-pd-data/releases/2021_v2-5release_0510/amp_pd_case_control.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/amp_pd_participant_wgs_duplicates.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/amp_pd_participants.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/rna_sample_inventory.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/wgs_gatk_joint_genotyping_samples.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/wgs_sample_inventory.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/


In [6]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_WGS_RELEASE_PATH }')

Executing: gsutil -u terra-ed19e231 ls gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs
gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/wgs_qc_flags.csv
gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/wgs_samples.csv
gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/gatk/
gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/plink/
gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/topmed/


In [7]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_CLINICAL_RELEASE_PATH}')

Executing: gsutil -u terra-ed19e231 ls gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_CSF_abeta_tau_ptau.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_CSF_abeta_tau_ptau_dictionary.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_CSF_beta_glucocerebrosidase.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_CSF_beta_glucocerebrosidase_dictionary.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_SomaLogic_plasma.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_SomaLogic_plasma_dictionary.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_other.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Biospecimen_analyses_other_dictionary.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/Caffei

In [25]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r /home/jupyter/notebooks/working_dir/PDBP_final.rds {WORKSPACE_BUCKET}/files')

Executing: gsutil -mu terra-ed19e231 cp -r /home/jupyter/notebooks/working_dir/PDBP_final.rds gs://fc-cd759889-2702-4f72-a832-0be756073417/files
CommandException: No URLs matched: /home/jupyter/notebooks/working_dir/PDBP_final.rds
CommandException: 1 file/object could not be transferred.


In [26]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/files')


Executing: gsutil -mu terra-ed19e231 ls gs://fc-cd759889-2702-4f72-a832-0be756073417/files
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/HapMapIII_CGRCh38.bed
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/HapMapIII_CGRCh38.bim
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/HapMapIII_CGRCh38.fam
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/HapMapIII_CGRCh38.log
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/HapMapIII_CGRCh38.snps
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PCA.eigenvec
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PCA_outliersAncestry_4SD.txt
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PDBP_QCed.PCA.eigenvec
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PDBP_QCed_plink1.PCA.eigenvec
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PDBP_final.rds
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PD_MDSUPDRSIII.csv
gs://fc-cd759889-2702-4f72-a832-0be756073417/files/SNPsDF_recodeA.raw
gs://fc-cd759889-2702-4f72-a832-0be756

## Extracting data

### Get the PD data
    

In [27]:
pd_case_control_df = gcs_read_csv(os.path.join(GS_RELEASE_PATH, 'amp_pd_case_control.csv'))
pd_case_control_df.info()
pd_case_control_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772 entries, 0 to 10771
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   participant_id                  10772 non-null  object
 1   diagnosis_at_baseline           10772 non-null  object
 2   diagnosis_latest                10772 non-null  object
 3   case_control_other_at_baseline  10772 non-null  object
 4   case_control_other_latest       10772 non-null  object
dtypes: object(5)
memory usage: 420.9+ KB


,participant_id,diagnosis_at_baseline,diagnosis_latest,case_control_other_at_baseline,case_control_other_latest
0,BF-1018,Essential Tremor,Essential Tremor,Other,Other
1,BF-1050,Other Neurological Disorder(s),Other Neurological Disorder(s),Other,Other
2,BF-1074,Other Neurological Disorder(s),Other Neurological Disorder(s),Other,Other
3,BF-1181,Other Neurological Disorder(s),Other Neurological Disorder(s),Other,Other
4,BF-1215,Essential Tremor,Essential Tremor,Other,Other


In [28]:
# Drop PPMI data since this has been analysed separatedly 
pd_case_control_df_noPPMI = pd_case_control_df[~pd_case_control_df["participant_id"].str.startswith("PP")]
print(f'Total raw data points excluding PPMI {pd_case_control_df_noPPMI.shape[0]}')
pd_case_control_df_noPPMI['diagnosis_latest'].unique()

Total raw data points excluding PPMI 8827


array(['Essential Tremor', 'Other Neurological Disorder(s)',
       "Possible Alzheimer's Disease", 'Parkinsonism',
       'Dementia With Lewy Bodies', 'Corticobasal Degeneration',
       'Olivopontocerebellar Atrophy', 'Vascular Parkinsonism',
       "Fahr's Syndrome", 'LBD', 'Idiopathic PD', "Parkinson's Disease",
       'Multiple System Atrophy', 'Progressive Supranuclear Palsy',
       'No PD Nor Other Neurological Disorder'], dtype=object)

In [29]:
# Subset and keep only latest diagnosis 
pd_case_control_df_noPPMI_latest = pd_case_control_df_noPPMI[['participant_id', 'diagnosis_latest', 'case_control_other_latest']]
pd_case_control_df_noPPMI_latest.columns = ['ID', 'LATEST_DX', 'CASE_CONTROL']

#print(pd_case_control_df_noPPMI_latest.value_counts())
print(pd_case_control_df_noPPMI_latest['LATEST_DX'].value_counts())

pd_case_control_df_noPPMI_latest.info()
pd_case_control_df_noPPMI_latest.head()

No PD Nor Other Neurological Disorder    3523
LBD                                      2614
Parkinson's Disease                      2392
Idiopathic PD                             118
Multiple System Atrophy                    62
Progressive Supranuclear Palsy             61
Essential Tremor                           28
Dementia With Lewy Bodies                   9
Corticobasal Degeneration                   7
Other Neurological Disorder(s)              4
Possible Alzheimer's Disease                3
Parkinsonism                                3
Olivopontocerebellar Atrophy                1
Vascular Parkinsonism                       1
Fahr's Syndrome                             1
Name: LATEST_DX, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8827 entries, 0 to 9982
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            8827 non-null   object
 1   LATEST_DX     8827 non-null   object
 2   C

,ID,LATEST_DX,CASE_CONTROL
0,BF-1018,Essential Tremor,Other
1,BF-1050,Other Neurological Disorder(s),Other
2,BF-1074,Other Neurological Disorder(s),Other
3,BF-1181,Other Neurological Disorder(s),Other
4,BF-1215,Essential Tremor,Other


In [30]:
# Grab the enrollment info so we know how each patient was recruited 
enrollment_df = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'Enrollment.csv'))
enrollment_subset_df = enrollment_df[['participant_id', 'study_arm']].copy()
enrollment_subset_df.columns = ['ID', 'ENROLL_STUDY_ARM']

print(enrollment_subset_df['ENROLL_STUDY_ARM'].value_counts(dropna=False))
enrollment_subset_df.info()

Healthy Control                3377
PD                             2987
LBD                            2521
Genetic Cohort Unaffected       403
Genetic Cohort PD               272
Genetic Registry Unaffected     245
Genetic Registry PD             196
Disease Control                 159
SWEDD                            77
Prodromal                        64
NaN                               3
Unknown                           1
Name: ENROLL_STUDY_ARM, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10305 entries, 0 to 10304
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                10305 non-null  object
 1   ENROLL_STUDY_ARM  10302 non-null  object
dtypes: object(2)
memory usage: 161.1+ KB


In [31]:
demo_enrollment_df = pd_case_control_df_noPPMI_latest.merge(enrollment_subset_df, on="ID", how="left")
demo_enrollment_df.info()
demo_enrollment_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9109 entries, 0 to 9108
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                9109 non-null   object
 1   LATEST_DX         9109 non-null   object
 2   CASE_CONTROL      9109 non-null   object
 3   ENROLL_STUDY_ARM  8357 non-null   object
dtypes: object(4)
memory usage: 355.8+ KB


,ID,LATEST_DX,CASE_CONTROL,ENROLL_STUDY_ARM
0,BF-1018,Essential Tremor,Other,Healthy Control
1,BF-1050,Other Neurological Disorder(s),Other,Healthy Control
2,BF-1074,Other Neurological Disorder(s),Other,Healthy Control
3,BF-1181,Other Neurological Disorder(s),Other,Healthy Control
4,BF-1215,Essential Tremor,Other,Healthy Control


In [32]:
# Coding for PD_PHENO in AMP data
    # 1=Healthy control with no neurological disorders
    # 2=Positive PD case not in a genetic enrichment study, prodromal, or a SWEDD
conditions = [
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "Healthy Control") & (demo_enrollment_df['LATEST_DX'] == "No PD Nor Other Neurological Disorder"),
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "Healthy Control") & (demo_enrollment_df['LATEST_DX'] == "Idiopathic PD"),
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "PD") & (demo_enrollment_df['LATEST_DX'] == "Parkinson's Disease"),
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "PD") & (demo_enrollment_df['LATEST_DX'] == "Idiopathic PD")
]
choices = [1,3,2,2]

demo_enrollment_df['PD_PHENO'] = np.select(conditions, choices, default=-9)
demo_enrollment_df['PD_PHENO'].value_counts(dropna=False)


## Add study as a column 
# Add cohort column (Numpy's NAN for anything missing - though nothing should be missing)
demo_enrollment_df['COHORT'] = np.where(demo_enrollment_df.ID.str.contains("LB-"), "LBD",
                                        np.where(demo_enrollment_df.ID.str.contains("PD-"), "PDBP",
                                        np.where(demo_enrollment_df.ID.str.contains("HB-"), "HBS",
                                        np.where(demo_enrollment_df.ID.str.contains("LC-"), "LCC",
                                        np.where(demo_enrollment_df.ID.str.contains("BF-"), "BIOFIND",
                                        np.where(demo_enrollment_df.ID.str.contains("SU-"), "SURE-PD3",
                                        np.where(demo_enrollment_df.ID.str.contains("SY-"), "STEADY-PD3", np.nan)))))))

print(demo_enrollment_df['COHORT'].value_counts(dropna=True))

demo_enrollment_df.info()
demo_enrollment_df.head()

LBD           4586
PDBP          1606
HBS           1189
LCC            638
STEADY-PD3     616
SURE-PD3       261
BIOFIND        213
Name: COHORT, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9109 entries, 0 to 9108
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                9109 non-null   object
 1   LATEST_DX         9109 non-null   object
 2   CASE_CONTROL      9109 non-null   object
 3   ENROLL_STUDY_ARM  8357 non-null   object
 4   PD_PHENO          9109 non-null   int64 
 5   COHORT            9109 non-null   object
dtypes: int64(1), object(5)
memory usage: 498.1+ KB


,ID,LATEST_DX,CASE_CONTROL,ENROLL_STUDY_ARM,PD_PHENO,COHORT
0,BF-1018,Essential Tremor,Other,Healthy Control,-9,BIOFIND
1,BF-1050,Other Neurological Disorder(s),Other,Healthy Control,-9,BIOFIND
2,BF-1074,Other Neurological Disorder(s),Other,Healthy Control,-9,BIOFIND
3,BF-1181,Other Neurological Disorder(s),Other,Healthy Control,-9,BIOFIND
4,BF-1215,Essential Tremor,Other,Healthy Control,-9,BIOFIND


In [33]:
# We subset PD data only
demo_enrollment_df_pdOnly = demo_enrollment_df[demo_enrollment_df.PD_PHENO.eq(2)]
print(f'{demo_enrollment_df_pdOnly.shape[0]} PD patients found in AMP-PD data\n\n')

# There are a few duplicates in this CS df.
len(demo_enrollment_df_pdOnly.ID.unique())

demo_enrollment_df_pdOnly.info()
demo_enrollment_df_pdOnly.head()

2514 PD patients found in AMP-PD data


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2514 entries, 2672 to 5462
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                2514 non-null   object
 1   LATEST_DX         2514 non-null   object
 2   CASE_CONTROL      2514 non-null   object
 3   ENROLL_STUDY_ARM  2514 non-null   object
 4   PD_PHENO          2514 non-null   int64 
 5   COHORT            2514 non-null   object
dtypes: int64(1), object(5)
memory usage: 137.5+ KB


,ID,LATEST_DX,CASE_CONTROL,ENROLL_STUDY_ARM,PD_PHENO,COHORT
2672,BF-1002,Idiopathic PD,Case,PD,2,BIOFIND
2673,BF-1003,Idiopathic PD,Case,PD,2,BIOFIND
2674,BF-1004,Idiopathic PD,Case,PD,2,BIOFIND
2675,BF-1006,Idiopathic PD,Case,PD,2,BIOFIND
2676,BF-1008,Idiopathic PD,Case,PD,2,BIOFIND


### Get PD samples from cohorts with MDS UPDRS III scores avail

In [34]:
MDSUPDRSIII_dict = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'MDS_UPDRS_Part_III_dictionary.csv'))
MDSUPDRSIII_dict.info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(MDSUPDRSIII_dict)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TableName      77 non-null     object
 1   ColumnName     77 non-null     object
 2   Title          77 non-null     object
 3   Description    77 non-null     object
 4   Required       77 non-null     bool  
 5   DataType       77 non-null     object
 6   DataTypeRange  35 non-null     object
 7   UniqueValues   38 non-null     object
dtypes: bool(1), object(7)
memory usage: 4.4+ KB


,TableName,ColumnName,Title,Description,Required,DataType,DataTypeRange,UniqueValues
0,MDS_UPDRS_Part_III.csv,participant_id,Study Subject ID,Study Subject ID,True,String,NaN,NaN
1,MDS_UPDRS_Part_III.csv,GUID,Subject ID,Global Unique ID (USUBJID),False,String,NaN,NaN
2,MDS_UPDRS_Part_III.csv,visit_name,Visit Name,"Visit name: M - in months, SC - screening visi...",True,String,NaN,NaN
3,MDS_UPDRS_Part_III.csv,visit_month,Visit Month,Numeric visit in months; for visits prior base...,False,Float,NaN,NaN
4,MDS_UPDRS_Part_III.csv,code_upd2301_speech_problems,Movement Disorder Society Unified Parkinson's ...,3.01 MDS-UPDRS - Speech Problems (UPD2301),False,Integer,0 - 4,NaN
5,MDS_UPDRS_Part_III.csv,code_upd2302_facial_expression,Movement Disorder Society Unified Parkinson's ...,3.02 MDS-UPDRS - Facial Expression (UPD2302),False,Integer,0 - 4,NaN
6,MDS_UPDRS_Part_III.csv,code_upd2303a_rigidity_neck,Movement Disorder Society Unified Parkinson's ...,3.03a MDS-UPDRS - Rigidity Neck (UPD2303A),False,Integer,0 - 4,NaN
7,MDS_UPDRS_Part_III.csv,code_upd2303b_rigidity_rt_upper_extremity,Movement Disorder Society Unified Parkinson's ...,3.03b MDS-UPDRS - Rigidity Right Upper Extremi...,False,Integer,0 - 4,NaN
8,MDS_UPDRS_Part_III.csv,code_upd2303c_rigidity_left_upper_extremity,Movement Disorder Society Unified Parkinson's ...,3.03c MDS-UPDRS - Rigidity Left Upper Extremit...,False,Integer,0 - 4,NaN
9,MDS_UPDRS_Part_III.csv,code_upd2303d_rigidity_rt_lower_extremity,Movement Disorder Society Unified Parkinson's ...,3.03e MDS-UPDRS - Rigidity Left Lower Extremity,False,Integer,0 - 4,NaN


In [35]:
MDSUPDRSIII_allData = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'MDS_UPDRS_Part_III.csv'))
MDSUPDRSIII_allData.head()

,participant_id,GUID,visit_name,visit_month,code_upd2301_speech_problems,code_upd2302_facial_expression,code_upd2303a_rigidity_neck,code_upd2303b_rigidity_rt_upper_extremity,code_upd2303c_rigidity_left_upper_extremity,code_upd2303d_rigidity_rt_lower_extremity,...,upd2317d_rest_tremor_amplitude_left_lower_extremity,upd2317e_rest_tremor_amplitude_lip_or_jaw,upd2318_consistency_of_rest_tremor,upd2da_dyskinesias_during_exam,upd2db_movements_interfere_with_ratings,code_upd2hy_hoehn_and_yahr_stage,upd2hy_hoehn_and_yahr_stage,upd23a_medication_for_pd,upd23b_clinical_state_on_medication,mds_updrs_part_iii_summary_score
0,BF-1001,PDNW781VHY,M0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Normal,Normal,Normal,No,NaN,0.0,Stage 0,NaN,NaN,0.0
1,BF-1002,PDCB969UGG,M0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Normal,Slight,Slight,No,NaN,1.0,Stage 1,NaN,NaN,15.0
2,BF-1002,PDCB969UGG,M0_5,0.5,1.0,1.0,0.0,1.0,1.0,1.0,...,Normal,Slight,Normal,No,NaN,1.0,Stage 1,NaN,NaN,16.0
3,BF-1003,PDLW805AHT,M0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,Normal,Normal,Slight,No,NaN,1.0,Stage 1,NaN,NaN,12.0
4,BF-1003,PDLW805AHT,M0_5,0.5,1.0,1.0,0.0,0.0,0.0,0.0,...,Normal,Slight,Moderate,No,NaN,2.0,Stage 2,NaN,NaN,24.0


In [36]:
MDSUPDRSIII_allData['visit_month'].unique()

array([  0. ,   0.5,  -1. ,  12. ,  18. ,  24. ,  30. ,  36. ,  42. ,
        48. ,   6. ,  60. ,  54. ,   3. ,   9. ,  72. ,  84. ,  96. ,
        -2. , 108. ])

In [40]:
# We make sure we remove duplicates from the CS data
MDSUPDRSIII_allData_pdOnly = demo_enrollment_df_pdOnly.drop_duplicates().merge(MDSUPDRSIII_allData, 
                                                             left_on=['ID'], right_on=['participant_id'],
                                                             how='inner')

# We explore cohorts with longitudinal data available
print(MDSUPDRSIII_allData_pdOnly[['COHORT', 'visit_month']].value_counts(dropna=True).sort_index())

cohorts_interest = ['PDBP', 'STEADY-PD3', 'SURE-PD3']
MDSUPDRSIII_allData_pdOnly_cohortsFilt = MDSUPDRSIII_allData_pdOnly[MDSUPDRSIII_allData_pdOnly['COHORT'].isin(cohorts_interest)]

MDSUPDRSIII_allData_pdOnly_cohortsFilt.info()
MDSUPDRSIII_allData_pdOnly_cohortsFilt.head()

print(f'\n Number of PD patients in AMP-PD with MDS-UPDRS III data available: {MDSUPDRSIII_allData_pdOnly_cohortsFilt.ID.nunique()}')
print(f'\n Number of data points available: {MDSUPDRSIII_allData_pdOnly_cohortsFilt.shape[0]}')


COHORT      visit_month
BIOFIND     0.0            118
            0.5            117
HBS         0.0            646
PDBP        0.0            873
            6.0            411
            12.0           413
            18.0           360
            24.0           346
            30.0           308
            36.0           275
            42.0           132
            48.0            94
            54.0             9
            60.0            26
STEADY-PD3  0.0            293
            36.0           278
SURE-PD3    0.0            261
            6.0            248
            12.0           235
            18.0           176
            24.0           194
            30.0           223
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5155 entries, 881 to 6035
Data columns (total 83 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  ----

## Get the 33 MDS UPDRS III data points needed for our analysis and reshaping data

In [43]:
MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt_earlyPD['visit_name'].shape

(4894,)

In [44]:
# We get the columns with the prefix code_, as they have data stored as integer.
cols_interest = ['ID','COHORT',"visit_month", "visit_name", "mds_updrs_part_iii_summary_score"]
filter_cols = [col for col in MDSUPDRSIII_allData_pdOnly_cohortsFilt.columns 
                        if ('code' in col) | (col in cols_interest) ]
# Exluding HY updrs III measure
filter_cols.remove("code_upd2hy_hoehn_and_yahr_stage")


MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt = MDSUPDRSIII_allData_pdOnly_cohortsFilt[filter_cols].reset_index(drop = True)
MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt.head()

# Filter out patients' time points of than 36 months from BL
MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt['visit_month'].unique()
MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt_earlyPD = MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt[MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt['visit_month'].le(36)].reset_index(drop = True)

display(MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt_earlyPD)


# Saving the data into a csv object
MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt_earlyPD.to_csv("gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PD_MDSUPDRSIII.csv", index=False)
MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt.to_csv("gs://fc-cd759889-2702-4f72-a832-0be756073417/files/PD_MDSUPDRSIII_allVisits.csv", index=False)

MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt_earlyPD.columns

,ID,COHORT,visit_name,visit_month,code_upd2301_speech_problems,code_upd2302_facial_expression,code_upd2303a_rigidity_neck,code_upd2303b_rigidity_rt_upper_extremity,code_upd2303c_rigidity_left_upper_extremity,code_upd2303d_rigidity_rt_lower_extremity,...,code_upd2315b_postural_tremor_of_left_hand,code_upd2316a_kinetic_tremor_of_right_hand,code_upd2316b_kinetic_tremor_of_left_hand,code_upd2317a_rest_tremor_amplitude_right_upper_extremity,code_upd2317b_rest_tremor_amplitude_left_upper_extremity,code_upd2317c_rest_tremor_amplitude_right_lower_extremity,code_upd2317d_rest_tremor_amplitude_left_lower_extremity,code_upd2317e_rest_tremor_amplitude_lip_or_jaw,code_upd2318_consistency_of_rest_tremor,mds_updrs_part_iii_summary_score
0,PD-PDAA503EF5,PDBP,M0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
1,PD-PDAA503EF5,PDBP,M12,12.0,2.0,1.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0
2,PD-PDAA503EF5,PDBP,M18,18.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,27.0
3,PD-PDAA503EF5,PDBP,M24,24.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
4,PD-PDAA503EF5,PDBP,M30,30.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4889,SY-PDZX943HWN,STEADY-PD3,M36,36.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4890,SY-PDZY968RFA,STEADY-PD3,M0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0
4891,SY-PDZY968RFA,STEADY-PD3,M36,36.0,0.0,1.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0
4892,SY-PDZZ260HUM,STEADY-PD3,M0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0


Index(['ID', 'COHORT', 'visit_name', 'visit_month',
       'code_upd2301_speech_problems', 'code_upd2302_facial_expression',
       'code_upd2303a_rigidity_neck',
       'code_upd2303b_rigidity_rt_upper_extremity',
       'code_upd2303c_rigidity_left_upper_extremity',
       'code_upd2303d_rigidity_rt_lower_extremity',
       'code_upd2303e_rigidity_left_lower_extremity',
       'code_upd2304a_right_finger_tapping',
       'code_upd2304b_left_finger_tapping',
       'code_upd2305a_right_hand_movements',
       'code_upd2305b_left_hand_movements',
       'code_upd2306a_pron_sup_movement_right_hand',
       'code_upd2306b_pron_sup_movement_left_hand',
       'code_upd2307a_right_toe_tapping', 'code_upd2307b_left_toe_tapping',
       'code_upd2308a_right_leg_agility', 'code_upd2308b_left_leg_agility',
       'code_upd2309_arising_from_chair', 'code_upd2310_gait',
       'code_upd2311_freezing_of_gait', 'code_upd2312_postural_stability',
       'code_upd2313_posture', 'code_upd2314_body_br

## Load the demographics data and explore the distribution for each cohort

In [22]:
demographics_df = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'Demographics.csv'))
display(demographics_df)

patients = list(MDSUPDRSIII_allData_pdOnly_cohortsFilt_varsFilt_earlyPD.ID.unique())
demographics_df_filtered = demographics_df[demographics_df['participant_id'].isin(patients)].reset_index(drop=True)
#print(f'\nNumber of patients after filtering {demographics_df_filtered.shape[0]}')


demographics_df_filtered['COHORT'] = np.where(demographics_df_filtered.participant_id.str.contains("PD-"), "PDBP",
                                             np.where(demographics_df_filtered.participant_id.str.contains("SU-"), "SURE-PD3",
                                             np.where(demographics_df_filtered.participant_id.str.contains("SY-"), "STEADY-PD3", np.nan)))

display(demographics_df_filtered)

,participant_id,GUID,visit_name,visit_month,age_at_baseline,sex,ethnicity,race,education_level_years
0,BF-1001,PDNW781VHY,M0,0,55,Male,Not Hispanic or Latino,White,12-16 years
1,BF-1002,PDCB969UGG,M0,0,66,Female,Not Hispanic or Latino,White,12-16 years
2,BF-1003,PDLW805AHT,M0,0,61,Male,Not Hispanic or Latino,White,12-16 years
3,BF-1004,PDKW284DYW,M0,0,62,Male,Not Hispanic or Latino,White,12-16 years
4,BF-1005,PDTM274KX6,M0,0,61,Female,Not Hispanic or Latino,White,12-16 years
...,...,...,...,...,...,...,...,...,...
11221,SY-PDZW544CEL,PDZW544CEL,M0,0,62,Male,Not Hispanic or Latino,White,Unknown
11222,SY-PDZX554VNB,PDZX554VNB,M0,0,74,Male,Not Hispanic or Latino,White,Unknown
11223,SY-PDZX943HWN,PDZX943HWN,M0,0,46,Male,Not Hispanic or Latino,White,Unknown
11224,SY-PDZY968RFA,PDZY968RFA,M0,0,41,Male,Not Hispanic or Latino,Multiracial,Unknown


,participant_id,GUID,visit_name,visit_month,age_at_baseline,sex,ethnicity,race,education_level_years,COHORT
0,PD-PDAA503EF5,PDAA503EF5,M0,0,70,Female,Not Hispanic or Latino,White,12-16 years,PDBP
1,PD-PDAB074CYQ,PDAB074CYQ,M0,0,65,Male,Not Hispanic or Latino,White,12-16 years,PDBP
2,PD-PDAB549YWB,PDAB549YWB,M0,0,57,Male,Not Hispanic or Latino,White,12-16 years,PDBP
3,PD-PDAB729HWD,PDAB729HWD,M0,0,65,Male,Not Hispanic or Latino,White,Greater than 16 years,PDBP
4,PD-PDAB762PA3,PDAB762PA3,M0,0,59,Female,Not Hispanic or Latino,White,12-16 years,PDBP
...,...,...,...,...,...,...,...,...,...,...
1422,SY-PDZW544CEL,PDZW544CEL,M0,0,62,Male,Not Hispanic or Latino,White,Unknown,STEADY-PD3
1423,SY-PDZX554VNB,PDZX554VNB,M0,0,74,Male,Not Hispanic or Latino,White,Unknown,STEADY-PD3
1424,SY-PDZX943HWN,PDZX943HWN,M0,0,46,Male,Not Hispanic or Latino,White,Unknown,STEADY-PD3
1425,SY-PDZY968RFA,PDZY968RFA,M0,0,41,Male,Not Hispanic or Latino,Multiracial,Unknown,STEADY-PD3


In [23]:
# Getting the numbers for different summary statistics
display(demographics_df_filtered.groupby(['COHORT','sex']).agg({'sex':'count'}))
display(demographics_df_filtered.groupby(['COHORT','ethnicity']).agg({'ethnicity':'count'}))
display(demographics_df_filtered.groupby(['COHORT', 'race']).agg({'race':'count'}))
display(demographics_df_filtered.groupby(['COHORT']).agg({'COHORT':'count'}))



def quantile25(x):
    return x.quantile(0.25)
def quantile50(x):
    return x.quantile(0.50)
def quantile75(x):
    return x.quantile(0.75)
grouped_data = demographics_df_filtered.groupby('COHORT')
grouped_data.agg({"age_at_baseline": ["count","mean","std","min", "max", quantile25, quantile50, quantile75]})


#grouped_data['sex'].agg({'Frequency':'count'})
#prueba.shape
#counts_samples = list(prueba.iloc[:,0])
#counts_samples

demo_data = demographics_df_filtered[['COHORT',"sex","ethnicity","race","age_at_baseline"]].groupby(['COHORT']).describe()
demo_data.iloc[:,]




sex
COHORT     sex        
PDBP       Female  316
           Male    557
STEADY-PD3 Female   90
           Male    203
SURE-PD3   Female  128
           Male    133

ethnicity
COHORT     ethnicity                        
PDBP       Hispanic or Latino             27
           Not Hispanic or Latino        806
           Unknown                        40
STEADY-PD3 Hispanic or Latino             10
           Not Hispanic or Latino        283
SURE-PD3   Hispanic or Latino              7
           Not Hispanic or Latino        249
           Unknown                         5

race
COHORT     race                                           
PDBP       American Indian or Alaska Native              2
           Asian                                        12
           Black or African American                    20
           Multiracial                                   3
           Unknown                                      11
           White                                       825
STEADY-PD3 American Indian or Alaska Native              1
           Asian                                         4
           Black or African American                     6
           Multiracial                                   5
           Native Hawaiian or Other Pacific Islander     1
           Other                                         4
           White                                       272
SURE-PD3   Asian                                         1
           Black or African American                     2
           Multiracial                                   2
           Other                                         2
           White                                       254

,COHORT
COHORT,
PDBP,873
STEADY-PD3,293
SURE-PD3,261


age_at_baseline                                                   
                     count       mean       std   min   25%   50%   75%   max
COHORT                                                                       
PDBP                 873.0  64.292096  9.138995  34.0  58.0  65.0  71.0  87.0
STEADY-PD3           293.0  62.191126  9.052604  32.0  56.0  63.0  68.0  83.0
SURE-PD3             261.0  62.524904  9.688432  33.0  56.0  64.0  69.0  86.0

# Getting plink files to retrieve WGS for the PDBP cohort only and to update sex information

In [25]:
PDBP = demographics_df_filtered[demographics_df_filtered['COHORT'] == "PDBP"]
PDBP['sex_plink'] = PDBP['sex'].apply(lambda x: 2 if x == "Female" else 1 if x == "Male" else 0) 
display(PDBP)

FIDIID = {"FID": PDBP["participant_id"], "IID": PDBP["participant_id"]}
FIDIID_toPlink = pd.DataFrame(FIDIID)
SEX_toPlink = pd.DataFrame({"FID": PDBP["participant_id"], "IID": PDBP["participant_id"], "SEX": PDBP['sex_plink']})
display(FIDIID_toPlink)
display(SEX_toPlink)


# Uncomment if not existing
# FIDIID_toPlink.to_csv('/home/jupyter/notebooks/working_dir/tokeep_FIDIID.txt', header=False, index=False, sep = "\t",
#                       mode = 'a')
# SEX_toPlink.to_csv('/home/jupyter/notebooks/working_dir/toUpdate_sex.txt', header=False, index=False, sep = "\t",
#                       mode = 'a')

print(FIDIID_toPlink.shape)
print(SEX_toPlink)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,participant_id,GUID,visit_name,visit_month,age_at_baseline,sex,ethnicity,race,education_level_years,COHORT,sex_plink
0,PD-PDAA503EF5,PDAA503EF5,M0,0,70,Female,Not Hispanic or Latino,White,12-16 years,PDBP,2
1,PD-PDAB074CYQ,PDAB074CYQ,M0,0,65,Male,Not Hispanic or Latino,White,12-16 years,PDBP,1
2,PD-PDAB549YWB,PDAB549YWB,M0,0,57,Male,Not Hispanic or Latino,White,12-16 years,PDBP,1
3,PD-PDAB729HWD,PDAB729HWD,M0,0,65,Male,Not Hispanic or Latino,White,Greater than 16 years,PDBP,1
4,PD-PDAB762PA3,PDAB762PA3,M0,0,59,Female,Not Hispanic or Latino,White,12-16 years,PDBP,2
...,...,...,...,...,...,...,...,...,...,...,...
868,PD-PDZX962WBC,PDZX962WBC,M0,0,57,Male,Not Hispanic or Latino,White,12-16 years,PDBP,1
869,PD-PDZY923RAG,PDZY923RAG,M0,0,65,Male,Not Hispanic or Latino,White,12-16 years,PDBP,1
870,PD-PDZZ075NW6,PDZZ075NW6,M0,0,64,Male,Not Hispanic or Latino,White,Greater than 16 years,PDBP,1
871,PD-PDZZ133PFJ,PDZZ133PFJ,M0,0,43,Female,Unknown,White,12-16 years,PDBP,2


,FID,IID
0,PD-PDAA503EF5,PD-PDAA503EF5
1,PD-PDAB074CYQ,PD-PDAB074CYQ
2,PD-PDAB549YWB,PD-PDAB549YWB
3,PD-PDAB729HWD,PD-PDAB729HWD
4,PD-PDAB762PA3,PD-PDAB762PA3
...,...,...
868,PD-PDZX962WBC,PD-PDZX962WBC
869,PD-PDZY923RAG,PD-PDZY923RAG
870,PD-PDZZ075NW6,PD-PDZZ075NW6
871,PD-PDZZ133PFJ,PD-PDZZ133PFJ


,FID,IID,SEX
0,PD-PDAA503EF5,PD-PDAA503EF5,2
1,PD-PDAB074CYQ,PD-PDAB074CYQ,1
2,PD-PDAB549YWB,PD-PDAB549YWB,1
3,PD-PDAB729HWD,PD-PDAB729HWD,1
4,PD-PDAB762PA3,PD-PDAB762PA3,2
...,...,...,...
868,PD-PDZX962WBC,PD-PDZX962WBC,1
869,PD-PDZY923RAG,PD-PDZY923RAG,1
870,PD-PDZZ075NW6,PD-PDZZ075NW6,1
871,PD-PDZZ133PFJ,PD-PDZZ133PFJ,2


(873, 2)
               FID            IID  SEX
0    PD-PDAA503EF5  PD-PDAA503EF5    2
1    PD-PDAB074CYQ  PD-PDAB074CYQ    1
2    PD-PDAB549YWB  PD-PDAB549YWB    1
3    PD-PDAB729HWD  PD-PDAB729HWD    1
4    PD-PDAB762PA3  PD-PDAB762PA3    2
..             ...            ...  ...
868  PD-PDZX962WBC  PD-PDZX962WBC    1
869  PD-PDZY923RAG  PD-PDZY923RAG    1
870  PD-PDZZ075NW6  PD-PDZZ075NW6    1
871  PD-PDZZ133PFJ  PD-PDZZ133PFJ    2
872  PD-PDZZ298XEH  PD-PDZZ298XEH    2

[873 rows x 3 columns]


In [27]:
PDBP["visit_name"].unique()

array(['M0'], dtype=object)

In [ ]:
%%bash
head /home/jupyter/notebooks/working_dir/tokeep_FIDIID.txt
cat /home/jupyter/notebooks/working_dir/amppd_PDBPsamples.log

In [ ]:
%%bash
du -h /home/jupyter/notebooks/